In [ ]:
pip install mediapipe opencv-python numpy

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

def compute_distance(point1, point2):
    """Calculate Euclidean distance between two hand landmarks."""
    return np.linalg.norm([point1.x - point2.x, point1.y - point2.y])

def recognize_gesture(hand_landmarks):
    """Detect specific hand gestures based on landmark positions."""
    lm = mp_hands.HandLandmark
    
    thumb_tip, thumb_ip = hand_landmarks.landmark[lm.THUMB_TIP], hand_landmarks.landmark[lm.THUMB_IP]
    index_tip, index_mcp = hand_landmarks.landmark[lm.INDEX_FINGER_TIP], hand_landmarks.landmark[lm.INDEX_FINGER_MCP]
    middle_tip, middle_mcp = hand_landmarks.landmark[lm.MIDDLE_FINGER_TIP], hand_landmarks.landmark[lm.MIDDLE_FINGER_MCP]
    ring_tip, pinky_tip = hand_landmarks.landmark[lm.RING_FINGER_TIP], hand_landmarks.landmark[lm.PINKY_TIP]
    
    fingertips = [index_tip, middle_tip, ring_tip, pinky_tip]
    
    if thumb_tip.y < thumb_ip.y and all(f.y > middle_mcp.y for f in fingertips):
        return "Thumbs Up"
    elif thumb_tip.y > thumb_ip.y and all(f.y > middle_mcp.y for f in fingertips):
        return "Thumbs Down"
    elif all(f.y < middle_mcp.y for f in fingertips) and thumb_tip.y < thumb_ip.y:
        return "Open Hand"
    elif all(f.y > middle_mcp.y for f in fingertips) and compute_distance(thumb_tip, index_tip) < 0.05:
        return "Fist"
    elif index_tip.y < index_mcp.y and all(f.y > middle_mcp.y for f in fingertips[1:]):
        return "Index Finger Up"
    elif index_tip.y < index_mcp.y and middle_tip.y < middle_mcp.y and all(f.y > middle_mcp.y for f in fingertips[2:]):
        return "Peace Sign"
    elif index_tip.y < index_mcp.y and pinky_tip.y < middle_mcp.y and all(f.y > middle_mcp.y for f in fingertips[1:3]):
        return "Rock Sign"
    elif compute_distance(thumb_tip, index_tip) < 0.05 and all(tip.y > middle_mcp.y for tip in [middle_tip, ring_tip, pinky_tip]):
        return "OK Sign"
    
    return "No Gesture Detected"

def main():
    """Start capturing video and detect hand gestures in real-time."""
    cap = cv2.VideoCapture(0)
    
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        
        frame = cv2.flip(frame, 1)  # Flip image for natural interaction
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                gesture = recognize_gesture(hand_landmarks)
                cv2.putText(frame, gesture, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                
                if gesture == "Fist":
                    cap.release()
                    cv2.destroyAllWindows()
                    return
        
        cv2.imshow('Hand Gesture Recognition', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


I0000 00:00:1741170972.731501 2332305 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1741170972.752965 2332443 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1741170972.761331 2332443 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-03-05 16:06:14.507 python[30045:2332305] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-05 16:06:14.507 python[30045:2332305] +[IMKInputSession subclass]: chose IMKInputSession_Modern
